In [1]:
import pandas as pd
import json
import getpass
import requests
import datetime
import os


In [2]:
WB_TOKEN = os.getenv('WB_TOKEN')
headers = {'Authorization': WB_TOKEN}

In [3]:
subj_chars_url = 'https://content-api.wildberries.ru/content/v2/object/charcs/1881'
resp = requests.get(subj_chars_url, headers=headers)


In [4]:
d = json.loads( resp.text)

In [5]:
d

{'data': [{'charcID': 12,
   'subjectName': 'Ленты упаковочные',
   'subjectID': 1881,
   'name': 'Рисунок',
   'required': False,
   'unitName': '',
   'maxCount': 1,
   'popular': True,
   'charcType': 1},
  {'charcID': 17596,
   'subjectName': 'Ленты упаковочные',
   'subjectID': 1881,
   'name': 'Материал изделия',
   'required': False,
   'unitName': '',
   'maxCount': 3,
   'popular': True,
   'charcType': 1},
  {'charcID': 59611,
   'subjectName': 'Ленты упаковочные',
   'subjectID': 1881,
   'name': 'Назначение подарка',
   'required': False,
   'unitName': '',
   'maxCount': 3,
   'popular': False,
   'charcType': 1},
  {'charcID': 59615,
   'subjectName': 'Ленты упаковочные',
   'subjectID': 1881,
   'name': 'Повод',
   'required': False,
   'unitName': '',
   'maxCount': 3,
   'popular': False,
   'charcType': 1},
  {'charcID': 88952,
   'subjectName': 'Ленты упаковочные',
   'subjectID': 1881,
   'name': 'Вес товара с упаковкой (г)',
   'required': False,
   'unitName': 'г'

In [6]:
names = [n['name'] + "->" + str(n['required']) for n in d['data']]

In [19]:
import math
import os
import re
import sqlite3

def gather_products_from_folder_and_db(db_path, images_folder):
    pattern = re.compile(r'^(\d+)-(\d+)\.jpg$', re.IGNORECASE)
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    products_list = []

    for filename in os.listdir(images_folder):
        match = pattern.match(filename)
        if not match:
            continue

        product_id, quantity_str = match.groups()
        cursor.execute("SELECT * FROM products WHERE product_id = ?", (product_id,))
        row = cursor.fetchone()

        if not row:
            continue

        (row_id, db_product_id, name, price_str, description, material,
         length_mm, width_mm, thickness_mkm, volume_l, cardboard_brand,
         cardboard_type, color, has_handles, box_shape, weight_g,
         transport_volume_m3, trade_mark, manufacturer_country) = row

        name = name.replace(',', '').replace('*', ' х ')

        options = []

        if material:
            options.append({
                "name": "Материал изделия",
                "value": material,
                "charc_type": 1
            })

        if length_mm:
            options.append({
                "name": "Длина ленты",
                "value": f"{length_mm} м",
                "charc_type": 1
            })

        if width_mm:
            options.append({
                "name": "Ширина ленты",
                "value": f"{width_mm} мм",
                "charc_type": 1
            })

        if color:
            options.append({
                "name": "Цвет",
                "value": color,
                "is_variable": True,
                "charc_type": 1,
                "variable_values": [color]
            })

        if manufacturer_country:
            options.append({
                "name": "Страна производства",
                "value": manufacturer_country,
                "charc_type": 1
            })

        if volume_l:
            options.append({
                "name": "Количество предметов в упаковке",
                "value": f"1 шт",
                "is_variable": True,
                "charc_type": 1,
                "variable_values": [f"1 шт"]
            })

        dimensions = {
            "length": 8,  # фиксированное значение 8 см
            "width": math.ceil(int(width_mm) / 10) if width_mm else 0,  # перевод мм в см с округлением вверх
            "height": 8   # фиксированное значение 8 см
        }

        quantity_str = 1
        variant = {
            "vendorCode": f"atllen_{product_id}_{quantity_str}",
            "title": name,
            "description": description or "Описание отсутствует",
            "options": options,
            "dimensions": dimensions
        }

        product_object = {
            "subjectID": 1881,  # для Ленты упаковочные
            "variants": [variant]
        }

        products_list.append(product_object)

    conn.close()
    return products_list


In [20]:

db_file = "lenta_atlasnaya.db"  # путь к вашей базе данных
folder_path = "atllen_images"  # путь к папке с изображениями

result = gather_products_from_folder_and_db(db_file, folder_path)
for item in result:
    print(item)
# Здесь item — уже готовая структура для Wildberries API.
# Дальше можно вызывать свой метод create_product_cards(result), как в предыдущем примере.


{'subjectID': 1881, 'variants': [{'vendorCode': 'atllen_9605315_1', 'title': 'Атласная лента жёлтая 15мм х 23м', 'description': 'Атласная лента жёлтого цвета, шириной 15 мм, намоткой 23 метра. Шелковые ленты предназначены для отделки и декорирования изделий из ткани, оформления праздничных мероприятий, упаковки подарков и цветов.\nОбласть применения:\nПодарочная упаковка;\nХэнд-мэйд поделки;\nФлористика;\nДекорирование.\nПреимущества:\nМягкость;\nСтойкость к выгоранию и выцветанию;\nДоступная цена.', 'options': [{'name': 'Материал изделия', 'value': 'Атласная лента', 'charc_type': 1}, {'name': 'Длина ленты', 'value': '23 м', 'charc_type': 1}, {'name': 'Ширина ленты', 'value': '15 мм', 'charc_type': 1}, {'name': 'Цвет', 'value': 'Жёлтый', 'is_variable': True, 'charc_type': 1, 'variable_values': ['Жёлтый']}, {'name': 'Страна производства', 'value': 'Китай', 'charc_type': 1}], 'dimensions': {'length': 8, 'width': 2, 'height': 8}}]}
{'subjectID': 1881, 'variants': [{'vendorCode': 'atllen_9

In [21]:
API_URL = "https://content-api.wildberries.ru/content/v2/cards/upload"



def create_product_cards(products):
  
    try:
        response = requests.post(API_URL, json=products, headers=headers)
        response_data = response.json()
        
        if response.status_code == 200 and not response_data.get("error", True):
            print("Карточки товаров успешно созданы.")
        else:
            print(f"Ошибка: {response_data.get('errorText', 'Неизвестная ошибка')}")
            print("Дополнительные ошибки:", response_data.get("additionalErrors", {}))
        
        return response_data
    except requests.RequestException as e:
        print(f"Ошибка при выполнении запроса: {e}")
        return None

In [22]:
result

[{'subjectID': 1881,
  'variants': [{'vendorCode': 'atllen_9605315_1',
    'title': 'Атласная лента жёлтая 15мм х 23м',
    'description': 'Атласная лента жёлтого цвета, шириной 15 мм, намоткой 23 метра. Шелковые ленты предназначены для отделки и декорирования изделий из ткани, оформления праздничных мероприятий, упаковки подарков и цветов.\nОбласть применения:\nПодарочная упаковка;\nХэнд-мэйд поделки;\nФлористика;\nДекорирование.\nПреимущества:\nМягкость;\nСтойкость к выгоранию и выцветанию;\nДоступная цена.',
    'options': [{'name': 'Материал изделия',
      'value': 'Атласная лента',
      'charc_type': 1},
     {'name': 'Длина ленты', 'value': '23 м', 'charc_type': 1},
     {'name': 'Ширина ленты', 'value': '15 мм', 'charc_type': 1},
     {'name': 'Цвет',
      'value': 'Жёлтый',
      'is_variable': True,
      'charc_type': 1,
      'variable_values': ['Жёлтый']},
     {'name': 'Страна производства', 'value': 'Китай', 'charc_type': 1}],
    'dimensions': {'length': 8, 'width': 2

In [23]:
resp = create_product_cards(result)

Карточки товаров успешно созданы.
